*1) Data Reduction:*

In [1]:
!pip install pyarrow
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
df_sb_f = pd.read_csv("data/df_timeseries_en.tsv.gz", compression="infer", sep="\t")
df_sb_f["datetime"] = pd.to_datetime(df_sb_f["datetime"])
df_sb_f.head(5)


,channel,category,datetime,views,delta_views,subs,delta_subs,videos,delta_videos,activity
0,UCBJuEqXfXTdcPSbGO9qqn1g,Film and Animation,2017-07-03,2.024946e+05,0.000000,650.222222,0.000000,5,0,3
1,UCBJuEqXfXTdcPSbGO9qqn1g,Film and Animation,2017-07-10,3.940857e+05,191591.111111,1046.000000,395.777778,6,1,1
2,UCBJuEqXfXTdcPSbGO9qqn1g,Film and Animation,2017-07-17,8.353938e+05,441308.083333,1501.500000,455.500000,6,0,1
3,UCBJuEqXfXTdcPSbGO9qqn1g,Film and Animation,2017-07-24,1.104577e+06,269183.250000,1750.000000,248.500000,6,0,0
4,UCBJuEqXfXTdcPSbGO9qqn1g,Film and Animation,2017-07-31,1.284406e+06,179828.600000,2008.300000,258.300000,6,0,0


In [8]:
print("channel count in TimeSeries table:", df_sb_f['channel'].nunique())
df_sb_f["category"].unique()

chanell count in TimeSeries table: 133516


array(['Film and Animation', 'Music', 'Comedy', 'Gaming',
       'Science & Technology', 'Sports', 'Entertainment', 'Education',
       'Nonprofits & Activism', 'People & Blogs', 'Howto & Style',
       'News & Politics', 'Travel & Events', 'Autos & Vehicles',
       'Pets & Animals', nan], dtype=object)

In [9]:
df_sb_f = df_sb_f.loc[df_sb_f["category"].isin(['Nonprofits & Activism', 'People & Blogs', 'News & Politics'])]
print("channel count in TimeSeries table after filteration:", df_sb_f['channel'].nunique())

chanell count in TimeSeries table after filteration: 21163


Now we save the reduced timeseries table

In [10]:
df_sb_f.to_pickle("data/df_timeseries_reduced.tsv.gz")

In [5]:
#too big.. don't fit in memory
#del df_sb_f
#df_vd_f = pd.read_feather("data/yt_metadata_helper.feather")
#df_vd_f["dummmy"] = 1
#df_vd_f.head(5)


In [11]:

df_ch_f = pd.read_csv("data/df_channels_en.tsv.gz", compression="infer", sep="\t")
df_ch_f["join_date"] = pd.to_datetime(df_ch_f["join_date"])
df_ch_f.head(5)


,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
0,Gaming,2010-04-29,UC-lHJZR3Gqxm24_Vd_AJ5Yw,PewDiePie,101000000,3956,3.0,2.087
1,Education,2006-09-01,UCbCmjCuTUZos6Inko4u57UQ,Cocomelon - Nursery ...,60100000,458,7.0,2.087
2,Entertainment,2006-09-20,UCpEhnqL0y41EpW2TvWAHD7Q,SET India,56018869,32661,8.0,2.087
3,Howto & Style,2016-11-15,UC295-Dw_tDNtZXFeAPAW6Aw,5-Minute Crafts,60600000,3591,9.0,2.087
4,Sports,2007-05-11,UCJ5v_MCY6GNUBTO8-D3XoAg,WWE,48400000,43421,11.0,2.087


In [13]:
print("channel count in TimeSeries table", df_ch_f['channel'].nunique())
df_ch_f['category_cc'].unique()

channel count in TimeSeries table 136470


array(['Gaming', 'Education', 'Entertainment', 'Howto & Style', 'Sports',
       'Music', 'Film and Animation', 'Comedy', 'Nonprofits & Activism',
       'People & Blogs', 'News & Politics', 'Science & Technology',
       'Pets & Animals', 'Autos & Vehicles', 'Travel & Events', nan],
      dtype=object)

In [15]:
df_ch_f = df_ch_f.loc[df_ch_f["category_cc"].isin(['Nonprofits & Activism', 'People & Blogs', 'News & Politics'])]
print("channel count in TimeSeries table after filteration:", df_ch_f['channel'].nunique())

channel count in TimeSeries table after filteration: 21645


In [16]:
df_ch_f.to_pickle("data/df_channels_reduced.tsv.gz")

In [28]:
from tqdm import tqdm
count = 0
count_after = 0
chunksize = 10 ** 5
flag=False
for chunk in tqdm(pd.read_json("data/yt_metadata_en.jsonl.gz",lines=True, chunksize=chunksize)):
    count += chunk['display_id'].nunique() 
    chunk = chunk.loc[chunk["categories"].isin(['Nonprofits & Activism', 'People & Blogs', 'News & Politics'])]
    chunk.drop(['crawl_date', 'categories'], axis=1)
    if not flag:
        flag=True
        chunk.to_csv('data/yt_metadata_reduced.tsv.gz')
    else:
        chunk.to_csv('data/yt_metadata_reduced.tsv.gz', mode='a', index=False, header=False)
    count_after += chunk['display_id'].nunique()

print("videos count: ", count)
print("videos count after reduction: ", count_after)


730it [34:45,  2.86s/it]

videos count:  72924794
videos count after reduction:  16569137


*2) Data Exploration & Processing:*